In [46]:
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms

In [47]:
train_set = torchvision.datasets.EMNIST(
    root='./data'
    ,split='byclass'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [48]:
train_loader = torch.utils.data.DataLoader(train_set 
          , batch_size = 10
          , shuffle = True )

In [49]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3)
        self.conv6 = nn.Conv2d(in_channels=30, out_channels=62, kernel_size=3)

        self.out = nn.AvgPool2d(kernel_size=3)

    def forward(self, t):

        # (1) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)

        # (2) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)

        # (1) max pool layer
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv3(t)
        t = F.relu(t)

        # (4) hidden conv layer
        t = self.conv4(t)
        t = F.relu(t)

        # (2) max pool layer
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (5) hidden conv layer
        t = self.conv5(t)
        t = F.relu(t)

        # (6) hidden conv layer
        t = self.conv6(t)
        t = F.relu(t)

        # (1) output layer
        t = self.out(t)
        t = F.softmax(t.view(-1, 62), dim=1)

        return t

In [50]:
import torch.optim as optim
torch.set_grad_enabled(True)

In [51]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [53]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(25):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

NameError: ignored